In [ ]:
import logging

import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s %(message)s",
)

In [1]:
bands = [
    "B01",
    "B02",
    "B03",
    "B04",
    "B05",
    "B06",
    "B07",
    "B08",
    "B8A",
    "B09",
    "B11",
    "B12"
]

spatial_resolution = 10

study_area = "CORDILLERA BLANCA"  # "CORDILLERA BLANCA" or "CORDILLERA VILCABAMBA"

# List of Sentinel-2 images to be resampled.
image_ids = [
    "S2A_MSIL2A_20160114T150652_N0201_R082_T18LYL_20160114T150919",
    "S2A_MSIL2A_20160612T150722_N0202_R082_T18LXL_20160612T151613",
    "S2A_MSIL2A_20160612T150722_N0202_R082_T18LYL_20160612T151613",
    "S2A_MSIL2A_20160702T150722_N0204_R082_T18LYL_20160702T151431",
    "S2A_MSIL2A_20160729T145732_N0204_R039_T18LYL_20160729T145736",
    "S2A_MSIL2A_20160831T150722_N0204_R082_T18LXL_20160831T151350",
    "S2A_MSIL2A_20160831T150722_N0204_R082_T18LYL_20160831T151350"
]

In [ ]:
def resample_image(_input_path, _output_path, _resolution, new_crs="EPSG:32718"):
    with rasterio.open(_input_path) as src:
        x_res = y_res = _resolution

        transform, width, height = calculate_default_transform(
            src.crs,
            new_crs,
            src.width,
            src.height,
            *src.bounds,
            resolution=(x_res, y_res),
        )

        kwargs = src.meta.copy()
        kwargs.update(
            {
                "crs": new_crs,
                "transform": transform,
                "width": width,
                "height": height,
                "driver": "GTiff"
            }
        )

        with rasterio.open(_output_path, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=new_crs,
                    resampling=Resampling.cubic
                )

In [ ]:
def get_bands(_images):
    _images_10m = [
        image
        for image in _images
        if any(
            _band in image.name for _band in ["B02_10m", "B03_10m", "B04_10m", "B08_10m"]
        )
    ]

    _images_20m = [
        image
        for image in _images
        if any(
            _band in image.name
            for _band in ["B05_20m", "B06_20m", "B07_20m", "B8A_20m", "B11_20m", "B12_20m"]
        )
    ]

    _images_60m = [
        image
        for image in _images
        if any(_band in image.name for _band in ["B01_60m", "B09_60m", "B10_60m"])
    ]

    return _images_10m, _images_20m, _images_60m

In [ ]:
for image_id in image_ids:
    for band in bands:
        input_path = f"data/{study_area}/{image_id}.SAFE"
        output_path = f"results/reproject/{study_area}/{image_id}"

        output_path.mkdir(parents=True, exist_ok=True)

        images = list(input_path.glob(f"**/*.jp2"))
        images = [image for image in images if any(band in image.name for band in bands)]
        logging.info("Found %s images", len(images))

        images_10m, images_20m, images_60m = get_bands(images)

        for img in images_10m:
            img_name = img.name
            logging.info(f"Resampling {img_name} to {spatial_resolution}m")
            resample_image(img,
                           output_path / img_name.replace("10m.jp2", f"{spatial_resolution}m_resampled.jp2").replace(
                               ".jp2", ".tif"), spatial_resolution)

        for img in images_20m:
            img_name = img.name
            logging.info(f"Resampling {img_name} to {spatial_resolution}m")
            resample_image(img,
                           output_path / img_name.replace("20m.jp2", f"{spatial_resolution}m_resampled.jp2").replace(
                               ".jp2", ".tif"), spatial_resolution)

        for img in images_60m:
            img_name = img.name
            logging.info(f"Resampling {img_name} to {spatial_resolution}m")
            resample_image(img,
                           output_path / img_name.replace("60m.jp2", f"{spatial_resolution}m_resampled.jp2").replace(
                               ".jp2", ".tif"), spatial_resolution)

logging.info("Done!")